In [ ]:
import numpy as np
from scipy import stats
import plotly.figure_factory as ff
import plotly.graph_objects as go

# Step 1: Define parameters
n_trials = 1000  # Number of trials
true_probabilities = [0.3, 0.4, 0.3]  # True probabilities for the categories
alpha_prior = np.array([1.1, 1.1, 1.1])  # Prior parameters for Dirichlet

# Step 2: Sample data from a multinomial distribution
observed_counts = np.random.multinomial(n_trials, true_probabilities)

# Step 3: Update the Dirichlet prior to get the posterior
alpha_posterior = alpha_prior + observed_counts

# Step 4: Generate grid points on the simplex
def generate_simplex_grid(n_points):
    p1 = np.linspace(0, 1, n_points)
    p2 = 1 - p1.copy()
    p_ = np.linspace(0, 1, n_points)
    p1 = (p1[np.newaxis, :] * p_[:, np.newaxis]).flatten()
    p2 = (p2[np.newaxis, :] * p_[:, np.newaxis]).flatten()
    p3 = 1 - p1 - p2
    mask = p3 >= 0
    return np.c_[p1[mask], p2[mask], p3[mask]]

n_points = 100
simplex_grid = generate_simplex_grid(n_points)

# Step 5: Compute densities for prior and posterior
dirichlet_prior = stats.dirichlet(alpha_prior)
dirichlet_posterior = stats.dirichlet(alpha_posterior)
prior_densities = dirichlet_prior.pdf(simplex_grid.T)
posterior_densities = dirichlet_posterior.pdf(simplex_grid.T)

# Step 6: Visualize prior and posterior distributions on the simplex
fig_prior = ff.create_ternary_contour(
    simplex_grid.T, prior_densities,
    pole_labels=['p1', 'p2', 'p3'],
    interp_mode='cartesian',
    showscale=True
)
fig_prior.update_layout(title='Dirichlet Prior Distribution')

fig_posterior = ff.create_ternary_contour(
    simplex_grid.T, posterior_densities,
    pole_labels=['p1', 'p2', 'p3'],
    interp_mode='cartesian',
    showscale=True
)
fig_posterior.update_layout(title='Dirichlet Posterior Distribution')

# Step 7: Visualize multinomial theoretical probabilities and sample
categories = ['Category 1', 'Category 2', 'Category 3']
sample_probs = observed_counts / n_trials

fig_multinomial = go.Figure()
fig_multinomial.add_trace(go.Bar(
    x=categories,
    y=true_probabilities,
    name='Theoretical Probabilities',
    marker_color='skyblue'
))
fig_multinomial.add_trace(go.Bar(
    x=categories,
    y=sample_probs,
    name='Sample Probabilities',
    marker_color='orange'
))
fig_multinomial.update_layout(
    title='Multinomial Distribution: Theoretical vs Sample',
    xaxis_title='Categories',
    yaxis_title='Probability',
    barmode='group',
    legend_title='Legend'
)

# Show plots
fig_prior.show()
fig_posterior.show()
fig_multinomial.show()




